In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [20]:
!pip install tenacity

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd "./drive/MyDrive/SemEval/"

/content/drive/MyDrive/SemEval


In [9]:
!ls

Chatgpt.ipynb  Cleaning.ipynb  Dev_Numerical_Reasoning.json  Train_Numerical_Reasoning.json


In [24]:
import os
import re
import json
import time
import random
import config
import pandas as pd
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_fixed
%run "Cleaning.ipynb"

In [25]:
# Define the function to fill the blank in the headline
@retry(stop=stop_after_attempt(3), wait=wait_fixed(20))
def fill_blank(news, masked_headline, prompt, client):
    # Construct the prompt by combining the news text and masked headline
    prompt = prompt.replace("<NEWS>", news).replace("<MASKED HEADLINE", masked_headline)
    messages = []
    messages.append(
        {"role": "system", "content": "You are a reasonable and accurate assistant"}
    )
    messages.append(
        {"role": "user", "content": prompt},
    )
    time.sleep(20)
    chat = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
    )
    reply = chat.choices[0].message.content
    # Extract the generated text from the response and return it
    return reply

In [13]:
# Load the dataset
with open("./data/Train_Numerical_Reasoning.json") as f:
    data = json.load(f)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

In [14]:
mask_token = "<MASK>"
df["news"] = df["news"].apply(replace_newline_tab)
df["news"] = df["news"].apply(remove_commas)
df["masked headline"] = df["masked headline"].apply(mask_text, args=(mask_token,))

In [15]:
df["masked headline"].head()

0    <MASK>K Walmart Part-Timers to Lose Health Ins...
1    Dax Shepard: Wedding to Kristen Bell Cost $<MASK>
2                          Nancy Reagan Dead at <MASK>
3    American Airlines Faces $<MASK>M Fine for Safe...
4    $<MASK>K Raised for Kids of Mom Dismembered on...
Name: masked headline, dtype: object

In [16]:
df["news"].head()

0    (Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walm...
1    (Oct 29, 2013  8:15 AM CDT) Dax Shepard and Kr...
2    (Mar 6, 2016  10:50 AM) Nancy Reagan, the help...
3    (Aug 15, 2008  5:11 AM CDT) American Airlines ...
4    (Apr 18, 2016  1:02 PM CDT) Ingrid Lyne, the S...
Name: news, dtype: object

In [73]:
def find_answer(text: str) -> str:
  text = text.replace(" ","")
  match = re.search(r"<MASK>=([0-9]*[/|\-|\\.]*[0-9]*)", text)
  if match:
      number = match.group(1)
      return number
  match = re.search(r"([0-9]+)", text)
  if match:
    number = match.group(1)
    return number
  return -1

In [34]:
def test_on_chatgpt(sample, prompt, client, fill_blank, find_answer):
  # Fill the blank in the headline for each item in the sample
  sample["response"] = sample.apply(
      lambda row: fill_blank(row["news"], row["masked headline"], prompt, client), axis=1
  )
  sample["predicted_ans"] = sample["response"].apply(find_answer)
  # Calculate the accuracy of ChatGPT's predictions
  accuracy = (sample["ans"] == sample["predicted_ans"]).mean()
  return accuracy

In [19]:
# Set up OpenAI API credentials
client = OpenAI(
    # This is the default and can be omitted
    api_key = config.API_KEY
)
# The prompt to feed into chatgpt
prompt = "I have a text of a news and its masked headline with mask token specified as <MASK>. The mask should be filled with a numerical value. you should just give me the numerical value to put instead of the mask. your response should be like this: <MASK> = <your predicted answer>\nthe news is:\n<NEWS>\nthe masked headline is:\n<MASKED HEADLINE>"

In [ ]:
# Sample 100 items from the dataset
sample = df.sample(n=100, random_state=42)
accuracy = test_on_chatgpt(sample, prompt, client, fill_blank, find_answer)
print(f"Accuracy: {accuracy:.2%}")

In [77]:
sample[['news', 'masked headline', 'ans', 'predicted_ans', 'calculation', 'response']].to_csv('chatgpt_output_2.csv', index=False)